In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import os
for dirname, _, filenames in os.walk('/kaggle/input/coronavirusdataset/'):
    print('>>> files in', dirname)
    print('>>>', filenames)
    pass


# # 나눔체 글꼴 설치 (matplotlib 한글폰트 깨지는 현상 해결)
# !apt-get update -qq
# !apt-get install fonts-nanum* -qq

>>> files in /kaggle/input/coronavirusdataset/
>>> ['TimeGender.csv', 'Case.csv', 'Region.csv', 'TimeProvince.csv', 'SearchTrend.csv', 'Policy.csv', 'PatientRoute.csv', 'SeoulFloating.csv', 'Time.csv', 'PatientInfo.csv', 'Weather.csv', 'TimeAge.csv']


## 1. '지역별 코로나 추이 데이터'와 '환자정보 데이터' 비교 - 지역별 환자정보 데이터의 대표성 확인
- 지역별 전체 통계가 반영된 TimeProvince와 지역별 일부 환자 정보가 누락된 PatientInfo를 비교해, 지역별로 PatientInfo 정보가 충분히 확보되었는지 확인하겠음

In [2]:
# 지역명 한글화 함수 정의
def translate_province(df):
    cities_eng = ['Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon', 'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do', 'Jeollabuk-do', 'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do', 'Korea']
    cities_kor = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '대한민국']
    cities_dict = dict(zip(cities_eng, cities_kor))
    df['province'] = df['province'].map(cities_dict)

In [3]:
# 환자정보 데이터 로드
PatientInfo_df = pd.read_csv('../input/coronavirusdataset/PatientInfo.csv')
translate_province(PatientInfo_df)  # 지역명 한글화

PatientInfo_df.tail()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
3321,7000000009,9651.0,female,NaN,20s,Korea,제주,Jeju-do,NaN,overseas inflow,NaN,NaN,14.0,NaN,2020-03-29,NaN,NaN,isolated
3322,7000000010,NaN,female,NaN,20s,Korea,제주,Jeju-do,NaN,overseas inflow,NaN,NaN,18.0,NaN,2020-04-03,NaN,NaN,isolated
3323,7000000011,NaN,male,NaN,30s,Korea,제주,Jeju-do,NaN,contact with patient,NaN,7.000000e+09,5.0,NaN,2020-04-03,NaN,NaN,isolated
3324,7000000012,NaN,female,NaN,20s,Korea,제주,Jeju-do,NaN,overseas inflow,NaN,NaN,9.0,NaN,2020-04-03,NaN,NaN,isolated
3325,7000000013,NaN,female,NaN,10s,China,제주,Jeju-do,NaN,overseas inflow,NaN,NaN,6.0,NaN,2020-04-14,NaN,NaN,isolated


In [4]:
# 지역별 코로나 추이 데이터 로드
TimeProvince_df = pd.read_csv('../input/coronavirusdataset/TimeProvince.csv')
translate_province(TimeProvince_df)  # 지역명 한글화

TimeProvince_df.tail()

,date,time,province,confirmed,released,deceased
1559,2020-04-20,0,전북,17,9,0
1560,2020-04-20,0,전남,15,8,0
1561,2020-04-20,0,경북,1361,1064,51
1562,2020-04-20,0,경남,116,87,0
1563,2020-04-20,0,제주,13,6,0


In [5]:
# '환자정보 데이터'에서 지역별 확진자 수 추출
city_patient_part = PatientInfo_df.groupby('province')['patient_id'].count().reset_index()


# '지역별 코로나 추이 데이터'에서 최근 날짜의 지역별 누적 확진자 수 추출
city_patient_full = TimeProvince_df[TimeProvince_df.date == TimeProvince_df.date.max()][['province', 'confirmed']]


# 지역별로 '환자정보 데이터'와 '지역별 코로나 추이 데이터'의 누적 확진자 수 비교
city_patient_df = pd.merge(city_patient_full, city_patient_part, how='left', on='province')
city_patient_df.columns = ['province', '실제확진자_수', '확진자정보제공_수']
city_patient_df['결측치_수'] = city_patient_df['실제확진자_수'] - city_patient_df['확진자정보제공_수']
city_patient_df['결측치_비율'] = (city_patient_df['결측치_수'] / city_patient_df['실제확진자_수'] * 100).\
                                    map(lambda x : x if x>=0 else 0).round(2)
city_patient_df.sort_values('결측치_비율', ascending=False).reset_index(drop=True).\
                style.background_gradient(cmap="Reds", subset=['결측치_비율'])


# '환자정보 데이터'에 결측치가 많아, 대체로 수가 약간 적게 나타남
# 단, 대구/강원은 '환자정보 데이터' 결측치가 지나치게 많고, 경기는 '환자정보 데이터'의 누적 확진자 수가 오히려 더 많은 것으로 나타남
# 결측치가 심하게 많은 대구/강원을 제외하고 지역별 분석을 시행하는 것이 타당하다고 판단됨

,province,실제확진자_수,확진자정보제공_수,결측치_수,결측치_비율
0,대구,6833,63,6770,99.08
1,강원,53,37,16,30.19
2,경북,1361,1223,138,10.14
3,울산,43,42,1,2.33
4,부산,132,129,3,2.27
5,충북,45,44,1,2.22
6,경남,116,115,1,0.86
7,서울,624,624,0,0
8,충남,141,141,0,0
9,전남,15,15,0,0


- '환자정보 데이터'에 결측치가 많아, 대체로 수가 약간 적게 나타남
- 단, 대구/강원은 '환자정보 데이터' 결측치가 지나치게 많고, 경기는 '환자정보 데이터'의 누적 확진자 수가 오히려 더 많은 것으로 나타남
- 결측치가 심하게 많은 대구/강원을 제외하고 지역별 분석을 시행하는 것이 타당하다고 판단됨

## 2. '지역정보 데이터'에 인구 수, 국민안심병원 수, 감염병전담병원 수 추가

In [6]:
# 지역정보 데이터 로드 (시/도 단위만 추출)
region_df = pd.read_csv('../input/coronavirusdatasetmodified/Region_translated.csv')
region_df = region_df[region_df.province_kor == region_df.city_kor].reset_index(drop=True).drop(columns=['code', 'city', 'city_kor'])

print('shape :', region_df.shape)
region_df.head()

shape : (18, 13)


,province,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,google_lat,google_lng,province_kor
0,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,37.566535,126.977969,서울특별시
1,Busan,35.179884,129.074796,304,408,22,1.40,18.41,8.6,6752,35.179554,129.075642,부산광역시
2,Daegu,35.872150,128.601783,229,355,11,1.62,15.78,7.5,5083,35.871435,128.601445,대구광역시
3,Gwangju,35.160467,126.851392,155,312,17,2.38,13.57,6.4,2852,35.159545,126.852601,광주광역시
4,Incheon,37.456188,126.705920,250,403,7,1.27,13.20,5.8,4497,37.456256,126.705206,인천광역시


## 2-1. 인구 수 추가

In [7]:
# (2020년 3월) 지역별, 연령대별 인구 수 데이터 로드 (출처 : http://27.101.213.4/)
pop_df = pd.read_csv('../input/koreanpopulationmar2020/Korean-population-Mar2020.csv',
                     encoding='cp949').drop(columns=['2020년03월_계_연령구간인구수'])


# 데이터 전처리
pop_df.columns = ['city_full', 'pop_total', 'pop_under10', 'pop_10s', 'pop_20s', 'pop_30s', 'pop_40s', 
                  'pop_50s', 'pop_60s', 'pop_70s', 'pop_80s', 'pop_90s', 'pop_over100']  # 컬럼명 변환
pop_df['province_kor'] = pop_df.city_full.str.split().map(lambda x : x[0])  # 시도명 추출
pop_df = pop_df.drop_duplicates('province_kor').drop(columns=['city_full']).reset_index(drop=True)  # 하위도시(시군구) 제외, 불필요 열 제외, 인덱스 초기화


# 데이터 전처리 - 대한민국(총합) 행 추가
pop_df.loc[:, :'pop_over100'] = pop_df.loc[:, :'pop_over100'].applymap(lambda x : x.replace(',','')).astype(int)  # 데이터타입 변환
pop_df.loc[ pop_df.shape[0] ] = pop_df.sum()
pop_df.iloc[-1, -1] = '대한민국'

print('shape :', pop_df.shape)
pop_df.head()

shape : (18, 13)


,pop_total,pop_under10,pop_10s,pop_20s,pop_30s,pop_40s,pop_50s,pop_60s,pop_70s,pop_80s,pop_90s,pop_over100,province_kor
0,9733655,655134,810349,1461212,1501662,1558128,1537114,1205699,691028,266097,40809,6423,서울특별시
1,3409932,240857,279909,435280,430494,516889,579584,513485,286808,110254,14551,1821,부산광역시
2,2433568,185945,233311,322594,302617,390628,428955,313213,171945,74745,8915,700,대구광역시
3,2952689,238763,279780,405826,416119,489213,515257,347493,172868,74768,11685,917,인천광역시
4,1455705,124957,162130,207787,190641,244621,234045,156074,91156,38293,5591,410,광주광역시


In [8]:
# 지역정보 데이터와 인구 수 데이터 병합
merged_region_df = pd.merge(region_df, pop_df, how='outer', on=['province_kor'])

print('shape :', merged_region_df.shape)
merged_region_df.head()

shape : (18, 25)


,province,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,...,pop_10s,pop_20s,pop_30s,pop_40s,pop_50s,pop_60s,pop_70s,pop_80s,pop_90s,pop_over100
0,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,...,810349,1461212,1501662,1558128,1537114,1205699,691028,266097,40809,6423
1,Busan,35.179884,129.074796,304,408,22,1.40,18.41,8.6,6752,...,279909,435280,430494,516889,579584,513485,286808,110254,14551,1821
2,Daegu,35.872150,128.601783,229,355,11,1.62,15.78,7.5,5083,...,233311,322594,302617,390628,428955,313213,171945,74745,8915,700
3,Gwangju,35.160467,126.851392,155,312,17,2.38,13.57,6.4,2852,...,162130,207787,190641,244621,234045,156074,91156,38293,5591,410
4,Incheon,37.456188,126.705920,250,403,7,1.27,13.20,5.8,4497,...,279780,405826,416119,489213,515257,347493,172868,74768,11685,917


## 2-2. 국민안심병원 수 추가

In [9]:
# 국민안심병원 목록 데이터 로드 (출처 : https://www.mohw.go.kr/react/popup_200128.html)
safe_hospitals_df = pd.read_excel('../input/covid-hospitals-data-200414/safe-hospitals.xlsx')


# 데이터 전처리
safe_hospitals_df = safe_hospitals_df[['시도','시군구','기관명','주소']]
safe_hospitals_df['province_kor'] = safe_hospitals_df['주소'].map(lambda x : x.split()[0])

print('전국 국민안심병원 수 :', safe_hospitals_df.shape[0])
safe_hospitals_df.head()

전국 국민안심병원 수 : 344


,시도,시군구,기관명,주소,province_kor
0,서울,강남구,연세대학교강남세브란스병원,서울특별시 강남구 언주로211,서울특별시
1,서울,강남구,강남차병원,서울특별시 강남구 논현로 566,서울특별시
2,서울,강남구,우리들병원,서울특별시 강남구 학동로 445,서울특별시
3,서울,강남구,강남베드로병원,"서울특별시 강남구 남부순환로2633, 2649",서울특별시
4,서울,강남구,나누리병원,서울특별시 강남구 언주로 731,서울특별시


In [10]:
# 시/도별 국민안심병원 수 계산
safe_hospitals_num = safe_hospitals_df.groupby('province_kor')['기관명'].count().reset_index().rename(columns={'기관명':'safe_hospitals_count'})

# 대한민국(총합) 행 추가
safe_hospitals_num.loc[ safe_hospitals_num.shape[0] ] = ['대한민국', safe_hospitals_num['safe_hospitals_count'].sum()]

print('shape :', safe_hospitals_num.shape)
safe_hospitals_num.head()

shape : (18, 2)


,province_kor,safe_hospitals_count
0,강원도,9
1,경기도,78
2,경상남도,29
3,경상북도,16
4,광주광역시,5


In [11]:
# 지역정보 데이터와 국민안심병원 수 데이터 병합
merged_region_df = pd.merge(merged_region_df, safe_hospitals_num, how='outer', on=['province_kor'])

print('shape :', merged_region_df.shape)
merged_region_df.head()

shape : (18, 26)


,province,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,...,pop_20s,pop_30s,pop_40s,pop_50s,pop_60s,pop_70s,pop_80s,pop_90s,pop_over100,safe_hospitals_count
0,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,...,1461212,1501662,1558128,1537114,1205699,691028,266097,40809,6423,67
1,Busan,35.179884,129.074796,304,408,22,1.40,18.41,8.6,6752,...,435280,430494,516889,579584,513485,286808,110254,14551,1821,33
2,Daegu,35.872150,128.601783,229,355,11,1.62,15.78,7.5,5083,...,322594,302617,390628,428955,313213,171945,74745,8915,700,17
3,Gwangju,35.160467,126.851392,155,312,17,2.38,13.57,6.4,2852,...,207787,190641,244621,234045,156074,91156,38293,5591,410,5
4,Incheon,37.456188,126.705920,250,403,7,1.27,13.20,5.8,4497,...,405826,416119,489213,515257,347493,172868,74768,11685,917,24


## 2-3. 감염병전담병원 수, 병상 수 추가

In [12]:
# 감염병 전담병원 목록 데이터 로드 (출처 : http://www.mohw.go.kr/react/al/sal0301vw.jsp?PAR_MENU_ID=04&MENU_ID=0403&page=1&CONT_SEQ=353539 (첨부파일을 전처리))
infection_hospitals_df = pd.read_csv('../input/covid-hospitals-data-200414/infection-hospitals.csv',
                                    index_col=0)


# 주소 이상치 확인
display(infection_hospitals_df.address.map(lambda x : x.split()[0]).value_counts())  # '대한민국'으로 시작하지 않는 데이터 1개
display(infection_hospitals_df.address.map(lambda x : x.split()[1]).value_counts())  # 세종특별자치시 없음, 시/도명 자리에 다른 값 있는 데이터 2개

display(infection_hospitals_df[ ~infection_hospitals_df.address.str.startswith('대한민국') ])
display(infection_hospitals_df[infection_hospitals_df.name.str.contains('세종') | infection_hospitals_df.address.str.contains('세종')])
display(infection_hospitals_df[infection_hospitals_df.address.str.startswith('대한민국 파주시')])

대한민국    67
Name: address, dtype: int64

대구광역시      7
경기도        7
경상북도       6
인천광역시      5
서울특별시      5
울산광역시      5
강원도        5
경상남도       4
대전광역시      4
충청남도       4
전라남도       3
전라북도       3
제주특별자치도    3
충청북도       2
광주광역시      2
부산광역시      1
세종특별자치시    1
Name: address, dtype: int64

,name,lat,lng,address,bed,province_kor


,name,lat,lng,address,bed,province_kor
29,NK세종병원,36.479452,127.262336,"대한민국 세종특별자치시 나성동 722, 연기군 남면 나성리 566",200,세종특별자치시


,name,lat,lng,address,bed,province_kor


In [13]:
# 데이터 전처리 - 이상치 처리
infection_hospitals_df.loc[20, 'address'] = '대한민국 대전광역시 유성구 자운로 90'
infection_hospitals_df.loc[29, 'address'] = '대한민국 세종특별자치시 나성동 722, 연기군 남면 나성리 566'
infection_hospitals_df.loc[33, 'address'] = '대한민국 경기도 파주시 금촌동 경기도립의료원파주병원'

# 데이터 전처리 - 시/도명 추출
infection_hospitals_df['province_kor'] = infection_hospitals_df['address'].map(lambda x : x.split()[1])

# 데이터 저장
infection_hospitals_df.to_csv('감염병전담병원_위치정보_2.csv')

print('전국 감염병 전담병원 수 :', infection_hospitals_df.shape[0])
print('전국 감염병 전담병원 병상 수 :', infection_hospitals_df.bed.sum())
infection_hospitals_df.head()

전국 감염병 전담병원 수 : 67
전국 감염병 전담병원 병상 수 : 25222


,name,lat,lng,address,bed,province_kor
0,국립중앙의료원,37.567241,127.005659,대한민국 서울특별시 중구 광희동 을지로 245,485,서울특별시
1,서울의료원,37.612772,127.098167,대한민국 서울특별시 중랑구 신내동 신내로 156,623,서울특별시
2,서남병원,37.511972,126.833222,대한민국 서울특별시 양천구 신정동 불광로20길 1 이펜,363,서울특별시
3,서북병원,37.604279,126.905087,대한민국 서울특별시 은평구 역촌동 갈현로7길 49,237,서울특별시
4,보라매병원,37.493337,126.924609,대한민국 서울특별시 동작구 신대방동 보라매로5길 20,827,서울특별시


In [14]:
infection_hospitals_df.groupby(['province_kor'])['name'].count().reset_index().rename(columns={'name':'infection_hospitals_count'})

,province_kor,infection_hospitals_count
0,강원도,5
1,경기도,7
2,경상남도,4
3,경상북도,6
4,광주광역시,2
5,대구광역시,7
6,대전광역시,4
7,부산광역시,1
8,서울특별시,5
9,세종특별자치시,1


In [15]:
# 시/도별 감염병 전담병원 수 계산
infection_hospitals_num = infection_hospitals_df.groupby(['province_kor'])['name'].count().reset_index().rename(columns={'name':'infection_hospitals_count'})

# 시/도별 감염병 전담병원 병상수 계산
infection_hospitals_bed_num = infection_hospitals_df.groupby(['province_kor'])['bed'].sum().reset_index().rename(columns={'bed':'infection_hospitals_bed_num'})

# 데이터프레임 병합, 대한민국(총합) 행 추가
hospital_numbers_df = pd.merge(infection_hospitals_num, infection_hospitals_bed_num, how='outer', on='province_kor')
hospital_numbers_df.loc[ hospital_numbers_df.shape[0] ] = ['대한민국', hospital_numbers_df['infection_hospitals_count'].sum(), hospital_numbers_df['infection_hospitals_bed_num'].sum()]

print('shape :', hospital_numbers_df.shape)
hospital_numbers_df.head()

shape : (18, 3)


,province_kor,infection_hospitals_count,infection_hospitals_bed_num
0,강원도,5,893
1,경기도,7,1359
2,경상남도,4,1027
3,경상북도,6,1388
4,광주광역시,2,357


In [16]:
# 지역정보 데이터와 감염병 전담병원 수 데이터 병합
merged_region_df = pd.merge(merged_region_df, hospital_numbers_df, how='outer', on=['province_kor'])

print('shape :', merged_region_df.shape)
merged_region_df.head()

shape : (18, 28)


,province,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,...,pop_40s,pop_50s,pop_60s,pop_70s,pop_80s,pop_90s,pop_over100,safe_hospitals_count,infection_hospitals_count,infection_hospitals_bed_num
0,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,...,1558128,1537114,1205699,691028,266097,40809,6423,67,5,2535
1,Busan,35.179884,129.074796,304,408,22,1.40,18.41,8.6,6752,...,516889,579584,513485,286808,110254,14551,1821,33,1,555
2,Daegu,35.872150,128.601783,229,355,11,1.62,15.78,7.5,5083,...,390628,428955,313213,171945,74745,8915,700,17,7,4236
3,Gwangju,35.160467,126.851392,155,312,17,2.38,13.57,6.4,2852,...,244621,234045,156074,91156,38293,5591,410,5,2,357
4,Incheon,37.456188,126.705920,250,403,7,1.27,13.20,5.8,4497,...,489213,515257,347493,172868,74768,11685,917,24,5,3071


In [17]:
# 최종 데이터프레임 저장
merged_region_df.to_csv('Region_sido_addPop_addHospital.csv', index=False)

# 저장상태 확인
test_df = pd.read_csv('Region_sido_addPop_addHospital.csv')
print('shape :', test_df.shape)
test_df.head()

shape : (18, 28)


,province,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,...,pop_40s,pop_50s,pop_60s,pop_70s,pop_80s,pop_90s,pop_over100,safe_hospitals_count,infection_hospitals_count,infection_hospitals_bed_num
0,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,...,1558128,1537114,1205699,691028,266097,40809,6423,67,5,2535
1,Busan,35.179884,129.074796,304,408,22,1.40,18.41,8.6,6752,...,516889,579584,513485,286808,110254,14551,1821,33,1,555
2,Daegu,35.872150,128.601783,229,355,11,1.62,15.78,7.5,5083,...,390628,428955,313213,171945,74745,8915,700,17,7,4236
3,Gwangju,35.160467,126.851392,155,312,17,2.38,13.57,6.4,2852,...,244621,234045,156074,91156,38293,5591,410,5,2,357
4,Incheon,37.456188,126.705920,250,403,7,1.27,13.20,5.8,4497,...,489213,515257,347493,172868,74768,11685,917,24,5,3071


In [18]:
# 인구수 시군구단위까지 처리코드 - 보류
# # (2020년 3월) 지역별, 연령대별 인구 수 데이터 로드 (출처 : http://27.101.213.4/)
# pop_df = pd.read_csv('../input/koreanpopulationmar2020/Korean-population-Mar2020.csv',
#                      encoding='cp949').drop(columns=['2020년03월_계_연령구간인구수'])

# # 데이터 전처리
# pop_df.columns = ['city_full', 'total', 'age_under10', 'age_10s', 'age_20s', 'age_30s', 'age_40s', 
#                   'age_50s', 'age_60s', 'age_70s', 'age_80s', 'age_90s', 'age_over100']  # 컬럼명 변환
# pop_df.loc[:, 'total':] = pop_df.loc[:, 'total':].applymap(lambda x : x.replace(',','')).astype(int)  # 데이터타입 변환

# pop_df['province_kor'] = pop_df.city_full.str.split().map(lambda x: x[0])  # 시/도명 분리
# pop_df['city_kor'] = pop_df['city_full'].str.split().map(lambda x: x[1] if not x[1].startswith('(') else x[0])  # 시/군/구명 분리
# pop_df = pop_df.drop(columns=['city_full']).drop_duplicates(['province_kor', 'city_kor']).iloc[:-2]  # 불필요 행/열 제외
# pop_df = pop_df.reset_index(drop=True)  # 인덱스 초기화

# # 데이터 전처리 - 대한민국(총합) 행 추가
# pop_df.loc[ pop_df.shape[0] ] = pop_df.sum()
# pop_df.iloc[-1, :-2] = (pop_df.iloc[-1, :-2] / 2).astype(int)  # 각 시/도마다 2번 더해지므로 2로 나눠줌
# pop_df.iloc[-1, -2:] = '대한민국'

# pop_df

In [19]:
# 국민안심병원 목록 시군구단위까지 처리코드 - 보류
# # 국민안심병원 목록 데이터 로드 (출처 : https://www.mohw.go.kr/react/popup_200128.html)
# safe_hospitals_df = pd.read_excel('../input/covid-hospitals-data-200414/safe-hospitals.xlsx')

# # 데이터 전처리
# safe_hospitals_df = safe_hospitals_df[['시도','시군구','기관명','주소']]
# safe_hospitals_df['province_kor'] = safe_hospitals_df['주소'].map(lambda x : x.split()[0])
# safe_hospitals_df['city_kor'] = safe_hospitals_df['주소'].map(lambda x : x.split()[1])
# safe_hospitals_df.loc[safe_hospitals_df.city_kor == '한누리대로', 'city_kor'] = '세종특별자치시'  # 이상치 처리
# safe_hospitals_df.loc[safe_hospitals_df.province_kor == '제주특별자치도', 'city_kor'] = '제주특별자치도'  # 다른 테이블과 통일
# safe_hospitals_df


# # 시/군/구별 국민안심병원 수 계산
# safe_hospitals_num = safe_hospitals_df.groupby(['province_kor','city_kor'])['기관명'].count().reset_index().rename(columns={'기관명':'safe_hospitals_count'})
# # 대한민국(총합) 행 추가
# safe_hospitals_num.loc[ safe_hospitals_num.shape[0] ] = ['대한민국', '대한민국', safe_hospitals_num['safe_hospitals_count'].sum()]
# safe_hospitals_num


# # 지역정보 데이터와 국민안심병원 수 데이터 병합
# test_df = pd.merge(region_df, safe_hospitals_num, how='left', on=['province_kor','city_kor'])
# # 결측치 0으로 변환
# test_df['safe_hospitals_count'] = test_df['safe_hospitals_count'].fillna(0).astype(int)
# test_df

In [20]:
# 감염병 전담병원 목록 주소 이상치 확인 - 보류
# [(name, addr) for name, addr in zip(infection_hospitals_df.name, infection_hospitals_df.address) if len(addr.split())<=4]

# [('영남대병원', '대한민국 대구광역시 대명동'),
#  ('빛고을전남대병원', '대한민국 광주광역시 덕남동 빛고을전남대병원'),
#  ('제2시립요양병원', '대한민국 광주광역시 남구 덕남동'),
#  ('파주병원', '대한민국 파주시 금촌동 파주병원'),
#  ('천안의료원', '대한민국 충청남도'),
#  ('국군대구병원(경산)', '대한민국 경상북도 경산시')]